# Pandas reference

Quick reference on getting common data processing tasks done with Pandas.

## Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

## Settings

### Maximum number of DF rows and columns to show

In [3]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

### Plot style

In [4]:
plt.style.use("ggplot")

## Basic concepts

## Reading data

## Inspecting data

## Cleaning data

### Long format to wide

### Wide format to long

## Transforming data

## Aggregating data

## Plotting data

Pandas plotting methods expect data to be in the wide format, with different series in different columns.

## Exporting data